#### École Polytechnique de Louvain - Université Catholique de Louvain
### LINMA1702 Modèles et méthodes d'optimisation I
# Optimisation d'un réseau de distribution d'eau <br>
 Année académique 2018-2019 <br>
 17 Mai 2019 <br>
 __Professeurs et assistants :__ <br>
 Prof. F. Glineur <br>
 Emilie Renard <br>
 <br>
 __Groupe 7 :__ <br>
 Sarah Glume - 2947 1200 <br>
 Ferdinand Hannequart - 7290 1600 <br>
 Augustin d'Oultremont - 2239 1700 <br>

## Introduction
 Dans le cadre de ce projet en modèles et méthodes d'optimisation, le fonctionnement d'un réseau de distribution d'eau dans une région montagneuse sera étudié, à l'aide des outils de l'optimisation linéaire.

 Dans un premier temps, un réseau existant sera analysé, ainsi que son coût de fonctionnement. Ensuite, des améliorations du réseau seront proposées, au moyen de la construction de châteaux d’eau. La dernière partie traitera de la conception d’un réseau optimal (topologie et dimensionnement des conduites).

### Données utilisées dans le code:
 - __P__ : la position des points (x,y,z) [m]
 - __A__ : la matrice d'incidence [-]
 - __alpha__ : constante [m/h]
 - __R__ : rayon des conduites (identique pour toutes les conduites) [m]

__Approvisionnement__
 - __A_pts__ : les indices des points d'approvisionnement [-]
 - __A_maxDeb__ : le débit maximal extractible en chaque point d'approvisionnement [m$^3$/h]
 - __A_cost__ : le coût d'extraction en chaque point d'approvisionnement [€/m$^3$]

__Consommation__
 - __C_pts__ : les indices des points de consommation [-]
 - __C_minDeb__ : le débit minimal consommable en consommable en chaque point de consommation [m$^3$/h]
 - __C_maxDeb__ : le débit maximal consommable en consommable en chaque point de consommation [m$^3$/h]
 - __C_price__ : le prix facturé (identique pour tous les points de consommation) [€/m$^3$]

__Intermédiaires__
 - __I_pts__ : les indices des points intermédiaires [-]

In [70]:
import numpy as np
from scipy.optimize import linprog
P = np.array([
 [-9.6 , 2.7 , 0.08394446],
 [-6.9 , 7.1 , 0.09969025],
 [-9.0 , 3.2 , 0.08757438],
 [-1.7 , 9.4 , 0.07859208],
 [-6.3 , 6.6 , 0.09795454],
 [-8.0 , 4.1 , 0.09254527],
 [-8.3 , 0.0 , 0.06546171],
 [-3.7 , 7.7 , 0.09001772],
 [-2.9 , 7.7 , 0.08740385],
 [-5.8 , 0.0 , 0.06384532],
 [-6.0 , 4.5 , 0.09171610],
 [-1.3 , 7.1 , 0.08500010],
 [-6.1 , 3.2 , 0.08564233],
 [-4.6 , 4.7 , 0.08941448],
 [-4.1 , 4.7 , 0.08835686],
 [-5.2 , 3.3 , 0.08463326],
 [-1.5 , 5.1 , 0.08664609],
 [-0.0 , 5.0 , 0.08933714],
 [-2.6 , 2.1 , 0.07603164],
 [-3.1 , 0.2 , 0.06363058],
 [-1.2 , 1.6 , 0.07571709],
 [-0.9 , 0.0 , 0.06600560],
 [-0.2 , 0.0 , 0.06870617]])

A = np.array([
    [ 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [-1, 0, 0,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0,-1, 0, 0, 0, 1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 1, 0, 0, 0, 0, 0,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0,-1, 0, 0,-1, 0, 0, 0, 0, 0,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0, 0, 1, 0,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,-1, 0, 0, 0, 0, 0, 1, 0, 0, 0,-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,-1,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,-1,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,-1,-1, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,-1,-1, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,-1,-1, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-1, 1, 0, 1],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,-1]])


alpha = 10**6
R = 1

A_pts = np.array([2, 6])
A_maxDeb = np.array([10000, 5000])
A_cost = np.array([0.2, 0.3])

C_pts = np.array([1, 4, 10, 20, 23])
C_minDeb = np.array([200, 200, 200, 200, 200])
C_maxDeb = np.array([4000,2000,2000,3000,2000])
C_price = 0.9

I_pts = np.array([3, 5, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22])

#### Calcul de valeurs, vecteurs et matrices utiles :

In [71]:
m = len(A)
n = len(A[0])

A_pts -= np.ones(len(A_pts), dtype=int)
C_pts -= np.ones(len(C_pts), dtype=int)
I_pts -= np.ones(len(I_pts), dtype=int)
dX = A.T @ P[:,0]
dY = A.T @ P[:,1]
dZ = A.T @ P[:,2]
length = np.sqrt(dX**2 + dY**2)
maxDeb = -(alpha*R*R*dZ/length)
C_prices = np.full(len(C_pts), C_price)

Prices = np.zeros(m) ; Prices[C_pts] = C_prices ; Prices[A_pts] = A_cost

## Partie 1 : Analyse d'un réseau existant
### Résolution numérique de la maximisation du bénéfice

In [72]:
c = Prices.T @ A

A_ub1 =  np.identity(n)
b_ub1 =  maxDeb
A_ub2 = -np.identity(n)
b_ub2 =  np.zeros(n)
A_ub3 =  A[C_pts]
b_ub3 =  C_maxDeb
A_ub4 = -A[C_pts]
b_ub4 = -C_minDeb
A_ub5 = -A[A_pts]
b_ub5 =  A_maxDeb
A_ub  = np.vstack(     [A_ub1,A_ub2,A_ub3,A_ub4,A_ub5])
b_ub  = np.concatenate([b_ub1,b_ub2,b_ub3,b_ub4,b_ub5])

A_eq = A[I_pts]
b_eq = np.zeros(len(I_pts))

x = linprog(-c, A_ub, b_ub, A_eq, b_eq); x.fun = -x.fun

theta = x.x / maxDeb
bilan = A @ x.x

#### Maximum :

In [73]:
print(x.fun)

6500.70165914608


#### Vecteur Theta (comment positionner les vannes) :

In [74]:
for i in range(len(theta)) :
    print(str(i+1) + " : " + str(theta[i]))

1 : 0.6731980400618953
2 : 0.0
3 : 0.9887576461039969
4 : 0.9437122137389621
5 : 0.6075683258396605
6 : 0.0
7 : 0.0
8 : 0.4722922193086017
9 : 0.0
10 : 0.0
11 : 0.6261055885946487
12 : 0.0
13 : 0.0
14 : 1.0
15 : 0.12441487636354756
16 : 0.6121191949102314
17 : 0.0
18 : 0.0
19 : 0.0
20 : 0.0
21 : 0.3780051392936077
22 : 0.0
23 : 0.0
24 : 0.0
25 : 1.0
26 : 0.0
27 : 0.0
28 : 0.6681240250965891
29 : 0.0
30 : 0.0
31 : 0.07044014258096072
32 : 0.534835894091108
33 : 0.0
34 : 1.0
35 : 0.0
36 : 0.0
37 : 0.0
38 : 0.056943419082826296
39 : 0.0


#### Bilan des débits en chaque point d'approvisionnement et de consommation :

In [75]:
print("Extraction aux points d'approvisionnement")
for i in A_pts :
    print("    " + str(i+1) + " : " + str(np.round(-bilan[i], 1)) + "   [m^3/h]")
print("Consommation aux points de consommation")
for i in C_pts :
    print("    " + str(i+1) + " : " + str(np.round( bilan[i], 1)) + "   [m^3/h]")

Extraction aux points d'approvisionnement
    2 : 5858.1   [m^3/h]
    6 : 4000.0   [m^3/h]
Consommation aux points de consommation
    1 : 4000.0   [m^3/h]
    4 : 2000.0   [m^3/h]
    10 : 646.6   [m^3/h]
    20 : 3000.0   [m^3/h]
    23 : 211.6   [m^3/h]


## Partie 2 : Améliorations du réseau

### Résolution numérique du dépassement de la demande conduisant à une diminution du prix

In [76]:
Prices_surpl = np.copy(Prices) ; Prices_surpl[C_pts] = Prices_surpl[C_pts] / 2
c1 = Prices      .T @ A
c2 = Prices_surpl.T @ A
c = np.concatenate([c1, c2])

A_ub1 =  np.hstack( [ np.identity(n), np.identity(n) ] )
b_ub1 =  maxDeb
A_ub2 = -np.hstack( [ np.identity(n), np.identity(n) ] )
b_ub2 =  np.zeros(n)
A_ub3a=  np.hstack( [ A[C_pts]      , np.zeros(A[C_pts].shape) ] )
b_ub3a=  C_maxDeb
A_ub3b=  np.hstack( [ np.zeros(A[C_pts].shape) , A[C_pts]      ] )
b_ub3b=  C_maxDeb*0.25
A_ub4 = -np.hstack( [ A[C_pts]      , A[C_pts]       ] )
b_ub4 = -C_minDeb
A_ub5 = -np.hstack( [ A[A_pts]      , A[A_pts]       ] )
b_ub5 =  A_maxDeb
A_ub  = np.vstack(     [A_ub1,A_ub2,A_ub3a,A_ub3b,A_ub4,A_ub5])
b_ub  = np.concatenate([b_ub1,b_ub2,b_ub3a,b_ub3b,b_ub4,b_ub5])

A_eq = np.hstack([ A[I_pts] , A[I_pts] ])
b_eq = np.zeros(len(I_pts))

x = linprog(-c, A_ub, b_ub, A_eq, b_eq); x.fun = -x.fun

theta = (x.x[:n] + x.x[n:]) / maxDeb
bilan = A @ (x.x[:n] + x.x[n:])

#### Maximum :

In [77]:
print(x.fun)

6963.20165914608


#### Vecteur Theta (comment positionner les vannes) :

In [78]:
for i in range(len(theta)) :
    print(str(i+1) + " : " + str(theta[i]))

1 : 0.841497550077369
2 : 0.215163135162942
3 : 0.9887576461039969
4 : 1.0
5 : 0.8231839598839734
6 : 0.2706481947311989
7 : 0.0
8 : 0.5903652741357521
9 : 0.0
10 : 0.0
11 : 0.663449703712147
12 : 0.0
13 : 0.0
14 : 1.0
15 : 0.12441487636354756
16 : 0.7651489936377892
17 : 0.0
18 : 0.0
19 : 0.0
20 : 0.0
21 : 0.3780051392936077
22 : 0.38397069868751904
23 : 0.0
24 : 0.0
25 : 1.0
26 : 0.0
27 : 0.0
28 : 0.9070517393368009
29 : 0.0
30 : 0.0
31 : 0.07044014258096072
32 : 0.668544867613885
33 : 0.0
34 : 1.0
35 : 0.0
36 : 0.0
37 : 0.0
38 : 0.056943419082826296
39 : 0.0


#### Bilan des débits en chaque point d'approvisionnement et de consommation :

In [79]:
print("Extraction aux points d'approvisionnement")
for i in A_pts :
    print("    " + str(i+1) + " : " + str(np.round(-bilan[i], 1)) + "   [m^3/h]")
print("Consommation aux points de consommation")
for i in C_pts :
    print("    " + str(i+1) + " : " + str(np.round( bilan[i], 1)) + "   [m^3/h]")

Extraction aux points d'approvisionnement
    2 : 7108.1   [m^3/h]
    6 : 5000.0   [m^3/h]
Consommation aux points de consommation
    1 : 5000.0   [m^3/h]
    4 : 2500.0   [m^3/h]
    10 : 646.6   [m^3/h]
    20 : 3750.0   [m^3/h]
    23 : 211.6   [m^3/h]
